In [1]:
import pdb

In [2]:
import sys

In [3]:
sys.path.append("../")

In [4]:
from helper.helper import *

In [5]:
import pandas as pd
import numpy as np
import os

## Variables

In [51]:
THRESHOLD = 10
month,start_date,end_date = ('june',11,17)
MLOPS_data_file_name = 'MLOPS Liveness June 11 - 17'
output_date = 'june11to17'
EXPORT_BOOL = True

## Reading

In [7]:
fraud = load_fraud_detection_data(month,start_date,end_date,fraud_column_names)

Data loaded for june 11
Data loaded for june 12
Data loaded for june 13
Data loaded for june 14
Data loaded for june 15
Data loaded for june 16
Data loaded for june 17


In [11]:
turing = load_mlops_data(MLOPS_data_file_name)

In [22]:
turing = turing.iloc[:,2:]

In [26]:
turing = turing.rename({'store_id':'storeid'},axis='columns')

In [27]:
turing = turing.rename({'type':'capture_type'},axis='columns')

## Manipulation

In [29]:
req_cols = ['xcall_id', 'response_code', 'orn','capture_type', 'is_live','manual_response', 'storeid']

In [30]:
turing.loc[(turing["digitalfake"]==True)|(turing['printfake']==True),'manual_response'] = 'fake'

In [31]:
turing_filtered = turing[turing['is_live']!='no']

In [32]:
df = pd.concat([fraud[req_cols],turing_filtered[req_cols]])

#### Get Journey status for the following orns

In [36]:
df['storeid'] = df['storeid'].apply(lambda x: clean_storeid(x))

In [34]:
df = filter_testing_orn(df)

In [37]:
new_df = df[df['manual_response']=='fake'].groupby('storeid').agg({'xcall_id': 'count', 'orn': ['nunique', 'unique']})

In [38]:
new_df.columns = ['count_of_fakeAttemtpts_from_store', 'count_of_faked_orns', 'faked_orns']

In [39]:
new_df.reset_index(inplace=True)

In [40]:
common_stores = new_df.sort_values('count_of_faked_orns',ascending=False)

In [41]:
common_stores['storeid'] = common_stores['storeid'].apply(lambda x: clean_storeid(x))

In [42]:
common_stores = common_stores[common_stores['count_of_fakeAttemtpts_from_store']>=THRESHOLD]

In [43]:
flagged_stores = load_blacklisted_data()

Blacklisted stores loaded from: ../data/flagged stores/3500_blacklisted_stores.csv


In [44]:
common_stores['is_store_blacklisted'] = False
common_stores.loc[common_stores['storeid'].isin(flagged_stores['storeid']),'is_store_blacklisted'] = True

In [45]:
common_stores.isna().sum()

storeid                              0
count_of_fakeAttemtpts_from_store    0
count_of_faked_orns                  0
faked_orns                           0
is_store_blacklisted                 0
dtype: int64

##### Save common_stores

In [61]:
save_df = common_stores[['storeid','is_store_blacklisted']]
export(save_df,"output/get_journey_status/",output_date+"_get_journey_status",EXPORT_BOOL)

##### Read Journey_status

In [302]:
os.listdir("input/revised_orns_journey/list1.csv")

['list1.csv', 'list2.csv']

In [335]:
journey_status = pd.DataFrame([])
for sheet in os.listdir("input/revised_orns_journey/"):
    print(sheet)
    if sheet!='.DS_Store':
        sheet_df = pd.read_csv("input/revised_orns_journey/"+sheet,header=None)
        print(len(sheet_df))
        sheet_df.columns = ['storeid', 'ORN','Journey']
        journey_status = pd.concat([journey_status,sheet_df])

    

list1.csv
2596
list2.csv
11266


In [336]:
pd.Series("NO0000HIDHIF").isin(journey_status['ORN'])

0    False
dtype: bool

In [337]:
# missed_orns = pd.read_csv('input/missed_orns.csv',header=None)
# missed_orns.columns = ['ORN','Journey']

# merged_data = pd.merge(missed_orns,df,how='inner',left_on = "ORN", right_on = 'orn')[['ORN','storeid','Journey']]

# merged_data = merged_data.drop_duplicates()

# journey_status = pd.concat([journey_status,merged_data])

In [338]:
# journey_status = pd.read_csv('journey status/journey_status_May25_June3.csv')

In [339]:
# journey_status.columns = ['ORN', 'storeid', 'Journey']

#### Check if we got all orns journey status

In [340]:
journey_status_requested_orns = pd.Series([st for row in common_stores["faked_orns"] for st in row])

In [341]:
journey_status_received_orns = journey_status['ORN']

In [342]:
len(journey_status_received_orns)

13862

In [345]:
len(journey_status_requested_orns)

1954

In [343]:
journey_status_requested_orns[np.logical_not(journey_status_requested_orns.isin(journey_status_received_orns))]

0       NO0000HIBSE0
1       NO0000HIDHIF
2       NO0000HIGD7X
3       NO0000HICDA4
4       NO0000HILAPM
            ...     
1949    NO0000HJIQIR
1950    NO0000HJOW79
1951    NO0000HILLXJ
1952    NO0000HI9OHM
1953    NO0000HJ1SPM
Length: 1572, dtype: object

In [344]:
# pd.DataFrame(temp.to_list()).to_csv('missed_ORNs.csv')

In [222]:
missed_data = sum(np.logical_not(journey_status_requested_orns.isin(journey_status_received_orns)))

In [223]:
if missed_data !=0:
    raise Exception("Some of the orns are missing")

In [224]:
missed_data

0

##### As, we have 0 False, it means we got the data for all the orns which we have sent

## Report

In [225]:
#Renaming journey values
journey_status['journey'] = 'Incomplete'
journey_status.loc[journey_status['Journey']=='Complete',"journey"]='Complete'

In [226]:
complete_incomplete_df = journey_status.groupby(['storeid','journey']).agg({'ORN':'unique'}).reset_index()
complete_orns = complete_incomplete_df[complete_incomplete_df['journey']=='Complete']
incomplete_orns = complete_incomplete_df[complete_incomplete_df['journey']=='Incomplete']

# total_ORNs_per_store = journey_status.groupby('storeid').agg({'ORN':'unique'}).reset_index()
# total_ORNs_per_store.columns = ['storeid', 'total_orns']

def get_intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

def get_string_list(df,list_cols):
    for col in list_cols:
        df.loc[:,col] = df[col].apply(lambda x: str(list(x)))
    return df


def get_report(df):
    storeid = df['storeid']
    try:
        complete_orns_list = complete_orns[complete_orns['storeid']==storeid]['ORN'].iloc[0]
    except:
        #IF THERE ARE NO COMPLETED ORNs FROM A GIVEN STORE
        complete_orns_list = []
    try:
        incomplete_orns_list = incomplete_orns[incomplete_orns['storeid']==storeid]['ORN'].iloc[0]
    except:
        #IF THERE ARE NO INCOMPLETED ORNs FROM A GIVEN STORE
        incomplete_orns_list = []
        
    df['list_complete_orn'] = complete_orns_list
    df['list_incomplete_orn'] = incomplete_orns_list
    df['count_completed'] = len(complete_orns_list)
    df['count_incompleted']= len(incomplete_orns_list)
    
    df['list_fake_complete'] = get_intersection(complete_orns_list,df['faked_orns'])
    df['list_fake_incomplete'] = get_intersection(incomplete_orns_list,df['faked_orns'])
    
    df['count_fake_complete'] = len(df['list_fake_complete']) 
    df['count_fake_incomplete'] = len(df['list_fake_incomplete'])
    
    completed_fake_not_detected_list = set(df['list_complete_orn']) - set(df['list_fake_complete']) #Gives real competed
    df['list_completed_fake_not_detected'] = completed_fake_not_detected_list
    df['count_completed_fake_not_detected'] = len(completed_fake_not_detected_list)

    return df

In [227]:
report = common_stores.apply(lambda x: get_report(x),axis=1)
list_cols = ['list_complete_orn', 'list_incomplete_orn','list_fake_complete', 'list_fake_incomplete','list_completed_fake_not_detected']
report = get_string_list(report,list_cols)

## Phase 2

#### Add if store is present in blacklist or not

In [236]:
flagged_stores = pd.read_csv('../data/flagged stores/3500_blacklisted_stores.csv')

In [357]:
flagged_stores_2 = pd.read_csv("../data/flagged stores/Reliance flagged stores - already blacklisted.csv")

In [358]:
flagged_stores_2.columns = ['storeid']

In [359]:
flagged_stores_3 = pd.read_csv("../data/flagged stores/Reliance flagged stores - to blacklist.csv")

In [360]:
flagged_stores_3.columns = ['storeid']

In [245]:
flagged_stores['storeid'] = flagged_stores['storeid'].apply(lambda x: clean_storeid(x))

In [361]:
flagged_stores_2['storeid'] = flagged_stores_2['storeid'].apply(lambda x: clean_storeid(x))

In [362]:
flagged_stores_3['storeid'] = flagged_stores_3['storeid'].apply(lambda x: clean_storeid(x))

In [264]:
report['Within 3500 Blacklisted stores'] = False
report.loc[report['storeid'].isin(flagged_stores['storeid']),'Within 3500 Blacklisted stores'] = True

In [366]:
len(flagged_stores_3)

2729

In [365]:
report['Within 1162 Blacklisted stores'] = False
report.loc[report['storeid'].isin(flagged_stores_2['storeid']),'Within 1162 Blacklisted stores'] = True

In [367]:
report['Within 2729 Blacklisted stores'] = False
report.loc[report['storeid'].isin(flagged_stores_3['storeid']),'Within 2729 Blacklisted stores'] = True

In [265]:
# report.to_csv('report_may25_june3.csv',index=False)

## Phase 3

#### Report with only required columns

In [371]:
report_cols = ['storeid','Within 3500 Blacklisted stores','Within 2729 Blacklisted stores','Within 1162 Blacklisted stores',"count_of_fakeAttemtpts_from_store",'count_fake_incomplete','count_fake_complete','count_completed_fake_not_detected']
          

In [372]:
final_report = report[report_cols]

In [275]:
final_report.columns = ['storeid','Within 3500 Blacklisted stores','Fraudlent attempts','Stopped ORNs','Successful (Fraudulent) ORNs','Successful ORNs with proper data']

In [373]:
final_report.columns = ['storeid','Within 3500 Blacklisted stores','Within 2729 Blacklisted stores','Within 1162 Blacklisted stores','Fraudlent attempts','Stopped ORNs','Successful (Fraudulent) ORNs','Successful ORNs with proper data']

In [374]:
sum(final_report['Successful ORNs with proper data'])

6967

In [375]:
final_report.to_csv('final_report_june4to10_2.csv',index=False)

## Phase 4

#### Get Individual lists of Fake Complete, Fake Incomplete, Fake Not Detected

In [278]:
date = 'june4_june10'

In [279]:
def get_list(df):
    list_orns = [st for row in df for st in row]
    return pd.DataFrame(list_orns)

In [280]:
def get_eval_list(df,list_cols):
    for col in list_cols:
        df[col] = df[col].apply(lambda x: eval(x))
    return df

In [281]:
report = get_eval_list(report,list_cols)

In [282]:
fake_complete_orns = get_list(report['list_fake_complete'])

In [283]:
fake_incomplete_orns = get_list(report['list_fake_incomplete'])

In [284]:
# fake_not_detected = get_list(report['list_completed_fake_not_detected'])

In [288]:
fake_complete_orns.to_csv('fraud_successful_orns_' + date + '.csv',index=False)

fake_incomplete_orns.to_csv('fraud_stopped_orns_' + date + '.csv',index=False)

# fake_not_detected.to_csv('fake_not_detected_orns_' + date + '.csv')

In [105]:
# may25-June3 Report
# 1162 stores


# friday - 204 stores